In [ ]:
import numpy as np
import faiss

def compute_weighted_average_embedding(embeddings, weights):
    """
    Compute the weighted average of embeddings.
    
    Args:
        embeddings (list or numpy.ndarray): A list or array of face embeddings.
        weights (list): A list of weights corresponding to each embedding.
        
    Returns:
        numpy.ndarray: The weighted average embedding.
    """
    embeddings = np.array(embeddings)
    weights = np.array(weights).reshape(-1, 1)  # Ensure weights are column-wise
    weighted_average = np.sum(embeddings * weights, axis=0) / np.sum(weights)
    return weighted_average

def compute_distance_based_selection(embeddings, reference_embedding):
    """
    Select the embedding that is closest to the reference embedding using faiss.
    
    Args:
        embeddings (list or numpy.ndarray): A list or array of face embeddings.
        reference_embedding (numpy.ndarray): The reference embedding to compare against.
        
    Returns:
        numpy.ndarray: The embedding closest to the reference.
    """
    embeddings = np.array(embeddings).astype('float32')
    reference_embedding = np.array(reference_embedding).reshape(1, -1).astype('float32')

    # Using Faiss to create an index and search for the nearest neighbor
    index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance index
    index.add(embeddings)
    _, closest_index = index.search(reference_embedding, 1)

    return embeddings[closest_index[0][0]]

def compute_clustered_embedding(embeddings):
    """
    Cluster the embeddings and get the centroid for more robustness.
    Uses K-Means to cluster and returns the centroid.
    
    Args:
        embeddings (list or numpy.ndarray): A list or array of face embeddings for left, front, and right profiles.
    
    Returns:
        numpy.ndarray: The centroid of the clustered embeddings.
    """
        # Convert embeddings to a numpy array and make sure it is float32 (required by faiss)
    embeddings = np.array(embeddings).astype('float32')
    
    if embeddings.shape[0] < 2:
        # If there's only one embedding, we return it directly
        return embeddings[0]

    # Number of clusters is 1 since we want a centroid of all the given embeddings
    n_clusters = 1
    d = embeddings.shape[1]  # Dimensionality of the embeddings

    # Create a Faiss KMeans object
    kmeans = faiss.Kmeans(d, n_clusters, niter=20, verbose=False)
    
    # Train the KMeans model using the embeddings
    kmeans.train(embeddings)
    
    # The centroid is in kmeans.centroids_
    centroid = kmeans.centroids[0]

    return centroid


# Main function to add a person to the database
def add_person_to_database(database, person_name, embeddings, method="clustered"):
    """
    Add a person to the database using different methods to compute the face embedding.
    
    Args:
        database (dict): The database storing face embeddings.
        person_name (str): The name of the person being added.
        embeddings (list or numpy.ndarray): A list or array of face embeddings for left, front, and right profiles.
        method (str): The method to compute the final embedding ("clustered", "weighted", "distance_based").
    """
    if method == "clustered":
        # Use K-Means clustering to find the centroid
        final_embedding = compute_clustered_embedding(embeddings)
    elif method == "weighted":
        # Weighted Average, assuming more importance to the front profile
        weights = [1, 2, 1]  # Assign higher weight to the front view
        final_embedding = compute_weighted_average_embedding(embeddings, weights)
    elif method == "distance_based":
        # Compute the average embedding and find the closest one
        average_embedding = np.mean(embeddings, axis=0)
        final_embedding = compute_distance_based_selection(embeddings, average_embedding)
    else:
        raise ValueError("Invalid method. Choose from 'clustered', 'weighted', or 'distance_based'.")
    
    # Add to the database
    database[person_name] = final_embedding
    print(f"Added {person_name} to the known faces database")

In [58]:
from pipresence.preprocess import ImagePreprocessor
from pipresence.config import Config
import os

Config.update_config(
    yolo_model_path = "../data/models/yolov8n-face.onnx",
    mobilefacenet_model_path = "../data/models/mobilefacenet_fixed.onnx"
)
embeddings = []
preprocessor = ImagePreprocessor()
imgs_path = "../data/images/elyor"
images = os.listdir(imgs_path)
for img in images:
    print(os.path.join(imgs_path, img))
    face, detections = preprocessor.process_input_image(os.path.join(imgs_path, img))
    embedding = preprocessor.recognizer.recognize_face(face)
    embeddings.append(embedding)

../data/images/elyor/right.jpg
../data/images/elyor/front.jpg
../data/images/elyor/left.jpg


In [59]:
imgs_path = "../data/test_images"
images = os.listdir(imgs_path)
for img in images:
    print(f"Reading {imgs_path}/{img}")
    face, detections = preprocessor.process_input_image(os.path.join(imgs_path, img))
    embedding = preprocessor.recognizer.recognize_face(face)
    sim_true, sim = preprocessor.recognizer.compare_embeddings(embeddings[1], embedding)
    print(f"Matches: {sim_true}, similarity: {sim}")

Reading ../data/test_images/ozodbek1.jpg
Matches: False, similarity: 0.023233618587255478
Reading ../data/test_images/ozodbek2.jpg
Matches: False, similarity: 0.12770265340805054
Reading ../data/test_images/idris_0.jpg
Matches: False, similarity: 0.362216055393219
Reading ../data/test_images/selena (2).jpg
Matches: False, similarity: 0.26621484756469727
Reading ../data/test_images/elyor1.jpg
Matches: True, similarity: 0.8635631799697876
Reading ../data/test_images/common.jpg
Matches: False, similarity: 0.3409087061882019
Reading ../data/test_images/janob_rasul.jpg
Matches: False, similarity: 0.37806740403175354


In [60]:

# Example data
database = {}
person_name = "Elyor"


# Add to database using different methods
add_person_to_database(database, person_name, embeddings, method="clustered")
add_person_to_database(database, person_name + " Weighted", embeddings, method="weighted")
add_person_to_database(database, person_name + " Distance Based", embeddings, method="distance_based")

# Check the database
print(database)

Added Elyor to the known faces database
Added Elyor Weighted to the known faces database
Added Elyor Distance Based to the known faces database
{'Elyor': array([-0.12488644, -0.05500732,  0.09350111, -0.03485516, -0.06635971,
       -0.13876937, -0.01464359,  0.03588411, -0.13525394,  0.05269077,
        0.09189095, -0.05842184,  0.0309348 , -0.09153684,  0.05107717,
       -0.04021215, -0.0337926 , -0.08296704,  0.06515458,  0.03370437,
        0.0022787 , -0.07802444,  0.02808763, -0.12195211, -0.00458466,
        0.03813045, -0.122896  ,  0.07874653, -0.1819981 ,  0.01344761,
       -0.18191129, -0.0350152 , -0.13791512, -0.05383592, -0.12305908,
        0.09306477, -0.027687  ,  0.08779693,  0.00410103, -0.06174715,
        0.04241253, -0.00829148, -0.05277452,  0.07616024,  0.02784561,
        0.0656791 , -0.05391767, -0.04395474, -0.02064281,  0.04058708,
       -0.11839603, -0.02991137, -0.00875236, -0.04066324, -0.02288179,
        0.09922622,  0.00778076,  0.06035806,  0.12235

WARNING clustering 3 points to 1 centroids: please provide at least 39 training points


In [61]:
imgs_path = "../data/test_images"
images = os.listdir(imgs_path)
for img in images:
    print(f"Reading {imgs_path}/{img}")
    face, detections = preprocessor.process_input_image(os.path.join(imgs_path, img))
    embedding = preprocessor.recognizer.recognize_face(face)
    sim_true0, sim0 = preprocessor.recognizer.compare_embeddings(database["Elyor"], embedding)
    print(f"Clustering matches: {sim_true0}, similarity: {sim0}")
    sim_true1, sim1 = preprocessor.recognizer.compare_embeddings(database["Elyor Weighted"], embedding)
    print(f"Weighted Avarage matches: {sim_true1}, similarity: {sim1}")
    sim_true2, sim2 = preprocessor.recognizer.compare_embeddings(database["Elyor Distance Based"], embedding)
    print(f"Distance Based matches: {sim_true2}, similarity: {sim2}")

Reading ../data/test_images/ozodbek1.jpg
Clustering matches: False, similarity: 0.06001235917210579
Weighted Avarage matches: False, similarity: 0.05138090573312887
Distance Based matches: False, similarity: 0.1131892204284668
Reading ../data/test_images/ozodbek2.jpg
Clustering matches: False, similarity: 0.22749382257461548
Weighted Avarage matches: False, similarity: 0.20561827095664553
Distance Based matches: False, similarity: 0.2601102888584137
Reading ../data/test_images/idris_0.jpg
Clustering matches: False, similarity: 0.3640337586402893
Weighted Avarage matches: False, similarity: 0.3722286832133632
Distance Based matches: False, similarity: 0.33150672912597656
Reading ../data/test_images/selena (2).jpg
Clustering matches: False, similarity: 0.1375933289527893
Weighted Avarage matches: False, similarity: 0.17607572823960504
Distance Based matches: False, similarity: 0.09393364191055298
Reading ../data/test_images/elyor1.jpg
Clustering matches: True, similarity: 0.8023427724838

In [8]:
from pipresence.tools.utils import draw_detections
import cv2

original_image = cv2.imread("/home/el02/PiPresence/data/test_images/ozodbek.jpg")
annotated = draw_detections(original_image, detections)

In [10]:
while True:
    cv2.imshow("exp",annotated)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()

In [1]:
from PIL import Image
import pyheif
import os

def convert_heic_to_jpg(heic_file_path, output_folder):
    # Read HEIC file
    heif_file = pyheif.read(heic_file_path)
    
    # Convert HEIC to PIL Image
    image = Image.frombytes(
        heif_file.mode, 
        heif_file.size, 
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )
    
    # Create output file path
    base_name = os.path.basename(heic_file_path)
    output_file_path = os.path.join(output_folder, os.path.splitext(base_name)[0] + ".jpg")
    
    # Save as JPG
    image.save(output_file_path, "JPEG")
    print(f"Converted {heic_file_path} to {output_file_path}")

# Example usage
heic_folder = "../data/images/muattar"
output_folder = "../data/images/muattar"
os.makedirs(output_folder, exist_ok=True)

heic_files = [f for f in os.listdir(heic_folder) if f.lower().endswith(".heic")]
for heic_file in heic_files:
    convert_heic_to_jpg(os.path.join(heic_folder, heic_file), output_folder)

Converted ../data/images/muattar/20241123_163258.heic to ../data/images/muattar/20241123_163258.jpg
Converted ../data/images/muattar/20241123_163249.heic to ../data/images/muattar/20241123_163249.jpg
Converted ../data/images/muattar/20241123_163238.heic to ../data/images/muattar/20241123_163238.jpg
